In [ ]:
! tar -xvf UTKFace.tar.gz

UTKFace/82_0_2_20170111210110290.jpg.chip.jpg
UTKFace/32_1_2_20170116180211518.jpg.chip.jpg
UTKFace/21_0_4_20170103225103768.jpg.chip.jpg
UTKFace/27_1_2_20170116171716287.jpg.chip.jpg
UTKFace/26_0_4_20170117153355923.jpg.chip.jpg
UTKFace/26_1_0_20170116231512359.jpg.chip.jpg
UTKFace/54_0_3_20170119211308521.jpg.chip.jpg
UTKFace/25_1_1_20170112235005249.jpg.chip.jpg
UTKFace/4_1_2_20161219211452533.jpg.chip.jpg
UTKFace/16_0_0_20170110232647979.jpg.chip.jpg
UTKFace/26_1_0_20170119192623081.jpg.chip.jpg
UTKFace/26_1_1_20170116170028717.jpg.chip.jpg
UTKFace/26_1_2_20170116180214166.jpg.chip.jpg
UTKFace/28_0_4_20170116220531686.jpg.chip.jpg
UTKFace/29_1_4_20170103230245713.jpg.chip.jpg
UTKFace/40_0_0_20170104204609691.jpg.chip.jpg
UTKFace/26_1_1_20170116160930991.jpg.chip.jpg
UTKFace/26_1_3_20170117153712926.jpg.chip.jpg
UTKFace/30_1_4_20170103183855083.jpg.chip.jpg
UTKFace/26_0_2_20170116175433114.jpg.chip.jpg
UTKFace/8_1_0_20170109204954253.jpg.chip.jpg
UTKFace/27_0_2_20161219192534211.jpg

In [ ]:
import os
from PIL import Image, ImageDraw
import pandas as pd
import numpy as np
from torch import nn, tensor, from_numpy
from torch.utils.data import Dataset, DataLoader
import torch
import random

In [ ]:
def coverImage(img):
    poly = []
    img_cpy = img.copy()
    for i in range(random.randrange(1,5)):
        for i in range(4):
            poly.append((random.randrange(200), random.randrange(200)))
        marked_img = ImageDraw.Draw(img_cpy).polygon(poly, fill="black")
    return img_cpy

def image2tensor(img):
    return from_numpy(np.array(img).transpose((2, 0, 1))).float()
    
def tensor2image(tnsr):
    return Image.fromarray(np.uint8(tnsr.numpy().transpose((1, 2, 0))))

def coverBatch(batch):
    covered_batch = batch.clone()
    for i, image in enumerate(covered_batch):
        covered_batch[i] = image2tensor(coverImage(tensor2image(image)))
    return covered_batch

class UtkFaceSet(Dataset):
    def __init__(self, root_dir):
        self.root = root_dir
        self.file_names = [name for name in os.listdir(root_dir) if os.path.isfile(root_dir+name)]
        self.length = len(self.file_names)
        
    def __len__(self):
        return self.length
    
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
            
        img_name = os.path.join(self.root + self.file_names[idx])
               
        return image2tensor(Image.open(img_name))
    

In [ ]:
class PartConvModel(nn.Module):
    
    def upconv2x2(self, in_channels, out_channels, mode='transpose'):
        if mode == 'transpose':
            return nn.ConvTranspose2d(
                in_channels,
                out_channels,
                kernel_size=2,
                stride=2)
        else:
            # out_channels is always going to be the same
            # as in_channels
            return nn.Sequential(
                nn.Upsample(mode='bilinear', scale_factor=2),
                self.conv1x1(in_channels, out_channels))
        
    def conv1x1(self, in_channels, out_channels, groups=1):
        return nn.Conv2d(
            in_channels,
            out_channels,
            kernel_size=1,
            groups=groups,
            stride=1)

    def __init__(self):
        super(PartConvModel, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1), nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1), nn.ReLU())
        
        self.layer2 = nn.Sequential(
            nn.MaxPool2d(kernel_size=2, stride=2),
            
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1), nn.ReLU(),
            nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1), nn.ReLU())
        
        self.layer3 = nn.Sequential(
            nn.MaxPool2d(kernel_size=2, stride=2),
        
            nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1), nn.ReLU(),
            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1), nn.ReLU())
        
        self.layer4 = nn.Sequential(
            nn.MaxPool2d(kernel_size=2, stride=2),
        
            nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1), nn.ReLU(),
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1), nn.ReLU())
        
        self.layer5 = nn.Sequential(
            nn.MaxPool2d(kernel_size=2, stride=2),
            
            nn.Conv2d(512, 1024, kernel_size=3, stride=1, padding=1), nn.ReLU(),
            nn.Conv2d(1024, 1024, kernel_size=3, stride=1, padding=1), nn.ReLU(),
            self.upconv2x2(1024, 512))
        
        self.layer6 = nn.Sequential(            
            nn.Conv2d(1024, 512, kernel_size=3, stride=1, padding=1), nn.ReLU(),
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1), nn.ReLU(),
            self.upconv2x2(512, 256))
        
        self.layer7 = nn.Sequential(            
            nn.Conv2d(512, 256, kernel_size=3, stride=1, padding=1), nn.ReLU(),
            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1), nn.ReLU(),
            self.upconv2x2(256, 128))
        
        self.layer8 = nn.Sequential(            
            nn.Conv2d(256, 128, kernel_size=3, stride=1, padding=1), nn.ReLU(),
            nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1), nn.ReLU(),
            self.upconv2x2(128, 64))
        
        self.layer9 = nn.Sequential(            
            nn.Conv2d(128, 64, kernel_size=3, stride=1, padding=1), nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1), nn.ReLU(),
            self.conv1x1(64, 3))
        
        
        
    def forward(self, x):
        out1 = self.layer1(x)
        out2 = self.layer2(out1)
        out3 = self.layer3(out2)
        out4 = self.layer4(out3)
        out5 = self.layer5(out4)
        
        copy4 = out4[:,:,0:out5.shape[2],0:out5.shape[3]]
        in6 = torch.cat((copy4, out5), 1)
        
        out6 = self.layer6(in6)
        
        copy3 = out3[:,:,0:out6.shape[2],0:out6.shape[3]]
        in7 = torch.cat((copy3, out6), 1)
        
        out7 = self.layer7(in7)
        
        copy2 = out2[:,:,0:out7.shape[2],0:out7.shape[3]]
        in8 = torch.cat((copy2, out7), 1)
        
        out8 = self.layer8(in8)
        
        copy1 = out1[:,:,0:out8.shape[2],0:out8.shape[3]]
        in9 = torch.cat((copy1, out8), 1)
        
        out9 = self.layer9(in9)
        
        return out9

In [ ]:
UTKFace_dir = './UTKFace/'

num_epochs = 5
num_classes = 10
batch_s = 100
learning_rate = 0.001

use_cuda = torch.cuda.is_available()
print("cuda avalible: ", use_cuda)
device = torch.device("cuda" if use_cuda else "cpu")
print(device)
model = PartConvModel().to(device)

utk_face = UtkFaceSet(UTKFace_dir)
utk_batch = DataLoader(utk_face, batch_size=batch_s)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Train the model
total_step = len(utk_batch)
loss_list = []
acc_list = []
for epoch in range(num_epochs):
    for i, (images) in enumerate(utk_batch):
        # Run the forward pass
        data = coverBatch(images)
        print(data.shape)
        if use_cuda:
          data = data.cuda()
        outputs = model(data)
        
        print(outputs.shape) 
        loss = criterion(data, images)
        loss_list.append(loss.item())

        # Backprop and perform Adam optimisation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Track the accuracy
        total = labels.size(0)
        _, predicted = torch.max(outputs.data, 1)
        correct = (predicted == labels).sum().item()
        acc_list.append(correct / total)

        if (i + 1) % 100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'
                  .format(epoch + 1, num_epochs, i + 1, total_step, loss.item(),
                          (correct / total) * 100))
        del data


cuda avalible:  True
cuda
torch.Size([100, 3, 200, 200])


RuntimeError: ignored